https://www.kaggle.com/rounakbanik/the-movies-dataset

In [20]:
import pandas as pd
import numpy as np

In [21]:
movies = pd.read_csv('movies.csv',index_col=0)
movies.head()

,title
id,
862,Toy Story
8844,Jumanji
15602,Grumpier Old Men
31357,Waiting to Exhale
11862,Father of the Bride Part II


In [22]:
ratings = pd.read_csv('ratings_shorter.csv',usecols=[1,2,3])
ratings.columns = ['user','id','rating']
ratings.head()

,user,id,rating
0,1,147,4.5
1,1,1221,5.0
2,1,1246,5.0
3,1,91542,5.0
4,2,1552,2.0


In [23]:
ratings = ratings.merge(movies,on='id').drop('id',axis=1)

In [24]:
ratings.head()

,user,rating,title
0,1,4.5,The 400 Blows
1,777,3.0,The 400 Blows
2,1784,2.5,The 400 Blows
3,1856,3.5,The 400 Blows
4,3019,4.0,The 400 Blows


In [25]:
table = pd.pivot_table(ratings,index='user',columns='title')

In [26]:
table.count(axis=1).sort_values(ascending=False).head(10)

user
8659      237
179792    219
45811     183
107720    168
270123    167
98415     164
243443    161
165352    160
229879    159
186059    156
dtype: int64

In [27]:
# precalculate some relevant data
process = pd.DataFrame(index=table.index)
process['count'] = table.count(axis=1)
process['length'] = np.sqrt((table.fillna(0)**2).sum(axis=1))
process

,count,length
user,,
1,2,6.726812
2,1,2.000000
3,1,2.000000
4,1,4.000000
5,2,6.403124
6,1,3.000000
7,6,9.420722
8,5,6.082763
9,1,2.500000


In [28]:
((table.loc[8659].fillna(0) * table.loc[179792].fillna(0))>0).sum()

18

In [29]:
(table.loc[8659].fillna(0) * table.loc[179792].fillna(0)).sum()

210.25

In [30]:
table.values.shape[1]

3008

In [28]:
# user based - find similar people and suggest what they liked

In [3]:
# item based - find correlations in the ratings data and suggest based on that